### Importing Libraries

In [231]:
import pandas as pd
import numpy as np
from sklearn.isotonic import IsotonicRegression

### Loading the Data


In [232]:
data = pd.read_csv('../Data/data.csv')

In [233]:
data = data [["Brand","Model","Year of Manufacture","Condition","Transmission","Fuel type","Vehicle_Price"]]

## Clean the 'Vehicle_Price' column
data['Vehicle_Price'] = data['Vehicle_Price'].replace({'Rs': '', ',': ''}, regex=True)
data['Vehicle_Price'] = pd.to_numeric(data['Vehicle_Price'], errors='coerce') 
data = data[data["Year of Manufacture"]>=1995] 

In [234]:
data

,Brand,Model,Year of Manufacture,Condition,Transmission,Fuel type,Vehicle_Price
0,Honda,CRV,2017,Used,Tiptronic,Petrol,19250000.0
1,Honda,CRV,2018,Used,Tiptronic,Petrol,22500000.0
2,Honda,Vezel,2024,Used,Tiptronic,Hybrid,22900000.0
3,Honda,Vezel,2018,Used,Tiptronic,Hybrid,14500000.0
4,Honda,Grace,2016,Used,Tiptronic,Hybrid,11250000.0
...,...,...,...,...,...,...,...
2790,BMW,740Le,2016,Used,Automatic,Petrol,45500000.0
2791,BMW,740e,2017,Used,Automatic,Petrol,50000000.0
2792,BMW,318i,2016,Used,Automatic,Petrol,21000000.0
2793,BMW,318i,2019,Used,Automatic,Petrol,26590000.0


In [235]:
data["Condition"].value_counts()

Condition
Used             1625
New               415
Reconditioned     263
Name: count, dtype: int64

In [236]:
## Get the count of each model
most_posted_model = data['Model'].value_counts().head(50).reset_index()

## Rename columns for clarity
most_posted_model.columns = ['Model', 'count']


In [237]:
## Get model counts
model_counts = most_posted_model['Model'].value_counts()

## Convert to lists
model_list = model_counts.index.tolist()


## Print the lists
print("Model:", model_list)


Model: ['Land Cruiser Prado', 'Dayz', 'Grand i10', 'Sunny', 'Sportage', 'Outlander', 'Lancer', 'Passo', 'Santa Fe', 'Wagon R Stingray', 'Eclipse Cross', 'Wagon R FZ', 'Voxy', 'Hilux', 'Hustler', 'Vezel', 'Celerio', 'Roomy', 'Leaf', 'Baleno', 'XBee', 'Accent', 'AD Wagon', 'Rexton', 'Sorento', 'Panda', 'Harrier', 'Raize', 'Land Cruiser Sahara', 'Yaris', 'Other Model', 'KUV 100', 'Alto', 'Vitz', 'Axio', 'Premio', 'Montero', 'Aqua', 'Maruti', 'CHR', 'Corolla', 'Mira', 'Wagon R', 'Swift', 'Prius', 'X-Trail', 'Wagon R FX', 'Allion', 'Spacia', 'March']


In [238]:
## get the top 50 models posted
data = data[data["Model"].isin(model_list)]

In [239]:
## Group by Vehicle_Info and calculate median price and count
median_df = data.groupby(['Brand','Model','Year of Manufacture','Transmission','Fuel type'])['Vehicle_Price'].agg(
    median_price='median',
    count='count'
).reset_index()

## Getting the min and max year for each model
table_1 = data.groupby(['Brand','Model','Transmission','Fuel type'])['Year of Manufacture'].agg(
    min_year='min',
    max_year='max'
).reset_index() 

## merge  two tables
table_2 = median_df.merge(table_1, on=["Brand", "Model", "Transmission", "Fuel type"], how='left')

In [240]:
median_df

,Brand,Model,Year of Manufacture,Transmission,Fuel type,median_price,count
0,Daihatsu,Mira,2006,Automatic,Petrol,4495000.0,1
1,Daihatsu,Mira,2015,Automatic,Petrol,5690000.0,3
2,Daihatsu,Mira,2016,Automatic,Petrol,5450000.0,3
3,Daihatsu,Mira,2017,Automatic,Petrol,5950000.0,3
4,Daihatsu,Mira,2023,Automatic,Petrol,6950000.0,23
...,...,...,...,...,...,...,...
685,Toyota,Yaris,2024,Automatic,Hybrid,16550000.0,5
686,Toyota,Yaris,2024,Automatic,Petrol,11370000.0,10
687,Toyota,Yaris,2025,Automatic,Hybrid,19000000.0,11
688,Toyota,Yaris,2025,Automatic,Petrol,18150000.0,10


In [241]:
table_2

,Brand,Model,Year of Manufacture,Transmission,Fuel type,median_price,count,min_year,max_year
0,Daihatsu,Mira,2006,Automatic,Petrol,4495000.0,1,2006,2025
1,Daihatsu,Mira,2015,Automatic,Petrol,5690000.0,3,2006,2025
2,Daihatsu,Mira,2016,Automatic,Petrol,5450000.0,3,2006,2025
3,Daihatsu,Mira,2017,Automatic,Petrol,5950000.0,3,2006,2025
4,Daihatsu,Mira,2023,Automatic,Petrol,6950000.0,23,2006,2025
...,...,...,...,...,...,...,...,...,...
685,Toyota,Yaris,2024,Automatic,Hybrid,16550000.0,5,2022,2025
686,Toyota,Yaris,2024,Automatic,Petrol,11370000.0,10,2007,2025
687,Toyota,Yaris,2025,Automatic,Hybrid,19000000.0,11,2022,2025
688,Toyota,Yaris,2025,Automatic,Petrol,18150000.0,10,2007,2025


In [242]:
## Generate a DataFrame with all the years from min_year to max_year for each model
all_years = []

## Loop through each row of table_2 to generate the full year range
for _, row in table_2.iterrows():
    years = list(range(row['min_year'], row['max_year'] + 1))  # +1 to include max_year
    for year in years:
        all_years.append({
            'Brand': row['Brand'],
            'Model': row['Model'],
            'Transmission': row['Transmission'],
            'Fuel type': row['Fuel type'],
            'Year of Manufacture': year,
            'min_year': row['min_year'],
            'max_year': row['max_year']
        })

## Create the new DataFrame
expanded_df = pd.DataFrame(all_years)
expanded_df = expanded_df.drop_duplicates(["Brand","Model","Transmission","Fuel type","Year of Manufacture"])


In [243]:
expanded_df

,Brand,Model,Transmission,Fuel type,Year of Manufacture,min_year,max_year
0,Daihatsu,Mira,Automatic,Petrol,2006,2006,2025
1,Daihatsu,Mira,Automatic,Petrol,2007,2006,2025
2,Daihatsu,Mira,Automatic,Petrol,2008,2006,2025
3,Daihatsu,Mira,Automatic,Petrol,2009,2006,2025
4,Daihatsu,Mira,Automatic,Petrol,2010,2006,2025
...,...,...,...,...,...,...,...
10084,Toyota,Yaris,Automatic,Hybrid,2023,2022,2025
10085,Toyota,Yaris,Automatic,Hybrid,2024,2022,2025
10086,Toyota,Yaris,Automatic,Hybrid,2025,2022,2025
10129,Toyota,Yaris,Tiptronic,Petrol,2023,2023,2023


In [244]:
## merge the all the expaded_df and the table_2
master_table = expanded_df.drop(["min_year","max_year"],axis=1).merge(table_2, on=["Brand", "Model","Year of Manufacture", "Transmission", "Fuel type"], how='left')

In [245]:
master_table

,Brand,Model,Transmission,Fuel type,Year of Manufacture,median_price,count,min_year,max_year
0,Daihatsu,Mira,Automatic,Petrol,2006,4495000.0,1.0,2006.0,2025.0
1,Daihatsu,Mira,Automatic,Petrol,2007,NaN,NaN,NaN,NaN
2,Daihatsu,Mira,Automatic,Petrol,2008,NaN,NaN,NaN,NaN
3,Daihatsu,Mira,Automatic,Petrol,2009,NaN,NaN,NaN,NaN
4,Daihatsu,Mira,Automatic,Petrol,2010,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1438,Toyota,Yaris,Automatic,Hybrid,2023,17900000.0,3.0,2022.0,2025.0
1439,Toyota,Yaris,Automatic,Hybrid,2024,16550000.0,5.0,2022.0,2025.0
1440,Toyota,Yaris,Automatic,Hybrid,2025,19000000.0,11.0,2022.0,2025.0
1441,Toyota,Yaris,Tiptronic,Petrol,2023,10450000.0,1.0,2023.0,2023.0


In [246]:
# Define the function to apply inflation logic
def apply_inflation(prev_price, year):
    if pd.isna(prev_price):
        return np.nan
    if year < 2005:
        if prev_price < 5_000_000:
            return prev_price * 1.03
        elif prev_price < 10_000_000:
            return prev_price * 1.04
        else:
            return prev_price * 1.05
    elif year < 2010:
        if prev_price < 5_000_000:
            return prev_price * 1.04
        elif prev_price < 10_000_000:
            return prev_price * 1.05
        else:
            return prev_price * 1.06
    elif year < 2015:
        if prev_price < 5_000_000:
            return prev_price * 1.05
        elif prev_price < 10_000_000:
            return prev_price * 1.06
        else:
            return prev_price * 1.07
    elif year < 2020:
        if prev_price < 5_000_000:
            return prev_price * 1.06
        elif prev_price < 10_000_000:
            return prev_price * 1.07
        else:
            return prev_price * 1.08
    elif year < 2025:
        if prev_price < 5_000_000:
            return prev_price * 1.07
        elif prev_price < 10_000_000:
            return prev_price * 1.08
        else:
            return prev_price * 1.08
    return prev_price

# Group and fill missing values
def fill_prices(group):
    for i in range(1, len(group)):
        if pd.isna(group.iloc[i]['median_price']):
            prev_price = group.iloc[i - 1]['median_price']
            year = group.iloc[i]['Year of Manufacture']
            group.iloc[i, group.columns.get_loc('median_price')] = apply_inflation(prev_price, year)
    return group

df = master_table.groupby(['Brand', 'Model', 'Transmission', 'Fuel type'], group_keys=False).apply(fill_prices)

df = df.drop(['min_year','max_year'],axis=1)




C:\Users\Pamith\AppData\Local\Temp\ipykernel_7536\3802801003.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = master_table.groupby(['Brand', 'Model', 'Transmission', 'Fuel type'], group_keys=False).apply(fill_prices)


In [247]:
# Assuming df is your DataFrame

def adjust_price(price):
    if price < 3_000_000:
        return price * 0.97
    elif price < 5_000_000:
        return price * 0.96
    elif price < 7_500_000:
        return price * 0.95
    elif price < 10_000_000:
        return price * 0.94
    else:
        return price * 0.93

# Apply the function
df['adjusted_median_price'] = df['median_price'].apply(adjust_price)


In [248]:
df

,Brand,Model,Transmission,Fuel type,Year of Manufacture,median_price,count,adjusted_median_price
0,Daihatsu,Mira,Automatic,Petrol,2006,4.495000e+06,1.0,4.315200e+06
1,Daihatsu,Mira,Automatic,Petrol,2007,4.674800e+06,NaN,4.487808e+06
2,Daihatsu,Mira,Automatic,Petrol,2008,4.861792e+06,NaN,4.667320e+06
3,Daihatsu,Mira,Automatic,Petrol,2009,5.056264e+06,NaN,4.803450e+06
4,Daihatsu,Mira,Automatic,Petrol,2010,5.359640e+06,NaN,5.091658e+06
...,...,...,...,...,...,...,...,...
1438,Toyota,Yaris,Automatic,Hybrid,2023,1.790000e+07,3.0,1.664700e+07
1439,Toyota,Yaris,Automatic,Hybrid,2024,1.655000e+07,5.0,1.539150e+07
1440,Toyota,Yaris,Automatic,Hybrid,2025,1.900000e+07,11.0,1.767000e+07
1441,Toyota,Yaris,Tiptronic,Petrol,2023,1.045000e+07,1.0,9.718500e+06


In [252]:
# --- 2. Prep dtypes ---
df['Year of Manufacture'] = df['Year of Manufacture'].astype(int)
df['count'] = df['count'].fillna(0).astype(float)

group_cols = ['Brand', 'Model', 'Transmission', 'Fuel type']

# --- Discount function ---
def apply_discount(price):
    if price < 3000000:
        return price * 0.95 
    elif price < 5000000:
        return price * 0.94
    elif price < 7500000:
        return price * 0.92 
    elif price < 10000000:
        return price * 0.91 
    elif price < 15000000:
        return price * 0.90     
    else:
        return price * 0.88

# --- 3 & 4. Group-wise adjustment ---
def adjust_group(group):
    group = group.sort_values('Year of Manufacture').copy()
    yrs    = group['Year of Manufacture'].values
    prices = group['median_price'].values
    weights= group['count'].values
    
    # Check if already non-decreasing
    if np.all(np.diff(prices) >= 0):
        group['adjusted_median_price'] = prices
    else:
        iso = IsotonicRegression(increasing=True, out_of_bounds='clip')
        group['adjusted_median_price'] = iso.fit_transform(
            X=yrs, y=prices, sample_weight=weights
        )
    return group

df = (
    df
    .groupby(group_cols, group_keys=False)
    .apply(adjust_group)
)

# --- 5. Apply discount to get final_price ---
df['final_price'] = df['adjusted_median_price'].apply(apply_discount)



C:\Users\Pamith\AppData\Local\Temp\ipykernel_7536\2036398010.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(adjust_group)


In [253]:
df.to_csv("output.csv")